In [16]:
import qiskit
qiskit.__version__

'1.1.1'

In [17]:
import numpy as np

# Importing standard Qiskit libralies
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_provider import IBMProvider
from qiskit.visualization import plot_histogram, plot_distribution
from qiskit.providers.fake_provider import GenericBackendV2, FakeQasmBackend
from qiskit_aer import StatevectorSimulator, AerSimulator
from qiskit import ClassicalRegister, QuantumRegister


In [18]:
# 結果表示用の文字列
state_label = ["|000>", "|001>", "|010>", "|000>", "|011>", "|100>", "|101>", "|110>", "|111>"]

In [19]:
# 量子回路の初期化
qr = QuantumRegister(3, "q")
cr0 = ClassicalRegister(1, "c0")
cr1 = ClassicalRegister(1, "c1")
circuit = QuantumCircuit(qr, cr0, cr1)

# ステップ1:量子もつれ状態を共有する
circuit.h(1) # アダマールゲート行列を適用
circuit.cx(1,2) # CNOTを適用
circuit.barrier()

# ステップ2:移動する量子ビットを準備する
# qr[0]を好きな量子状態に変化させてください
# たとえば|1>にしたい場合は次の命令を実行します
# circuit.x(0)

# ステップ3:アリスの手元の量子ビットにCNOTとHを適用する
circuit.cx(0,1)
circuit.h(0)
circuit.barrier()

# ステップ4:アリスの手元の量子ビットを測定し、得た値を古典ビットとしてボブに通信する
circuit.measure([0,1], [0,1])
circuit.barrier()

# ステップ5:得た古典ビットにより、ボブが手元の量子状態を変化させる
circuit.x(2).c_if(cr1,1)
circuit.x(2).c_if(cr0,1)

# 実行と結果取得(statevector_simulatorを利用)
backend = StatevectorSimulator()
result = backend.run(circuit).result().get_statevector(circuit, decimals=3)
#job = execute(circuit, backend)
#result = job.result()

#statevector = result.get_statevector(circuit)
print([str(coef) + label for (coef, label) in zip(result, state_label)])

['0j|000>', '0j|001>', '(1+0j)|010>', '0j|000>', '0j|011>', '0j|100>', '0j|101>', '0j|110>']
